In [10]:
## Initiales Importieren. Bevor das getan wird: `./panda.py categorize` ausführen!
import datetime
import pandas as pd
from panda import load_pc, add_cat
pc = load_pc()
pc = add_cat(pc)
pc.columns

Index(['account', 'book_date', 'valuta_date', 'party', 'book_text', 'purpose',
       'amount', 'balance', 'transfer_category', 'cat'],
      dtype='object')

In [13]:
## Finden von nicht-kategorisierten Abbuchungen auf den Konten 'giro', 'gesa' und 'common' im Jahr 2024

# Kopie des DataFrames erstellen
df = pc.loc[(pc.book_date.dt.year == 2024) & (pc.transfer_category.isna())]
df = df.copy()

# 'amount_type' setzen basierend auf dem Betrag
df.loc[df['amount'] > 0, 'amount_type'] = 'Gutschrift'
df.loc[df['amount'] <= 0, 'amount_type'] = 'Abbuchung'

# Relevante Spalten auswählen
cols = ['account', 'book_date', 'party', 'purpose', 'amount', 'cat']

# Filter für Abbuchungen und Konten
df = df.loc[
    (df['amount_type'].isin(['Abbuchung', 'Gutschrift'])) & (df['account'].isin(['giro', 'gesa', 'common']))
    ][cols].sort_values(by='amount', ascending=True)

# Nur Einträge ohne Kategorie anzeigen
filtered_df = df.loc[df['cat'].isna()]
print(filtered_df['amount'].sum())
filtered_df

-25400.55


,account,book_date,party,purpose,amount,cat
6880,gesa,2024-06-04,VISA AVOCADO STORE GMBH,NR XXXX 5021 HAMBURG DE KAUFUMSATZ 02.06 1402....,-1402.69,NaN
6408,gesa,2024-04-09,VISA SCHUHHAUS POTTHOFF,NR XXXX 5021 GUETERSLOH KAUFUMSATZ 05.04 14440...,-514.34,NaN
6640,gesa,2024-05-07,PayPal Europe S.a.r.l. et Cie S.C.A,"1034161056724/PP.7941.PP/. DB Vertr ieb GmbH, ...",-492.10,NaN
6089,gesa,2024-02-22,Waschbaer GmbH,K. 040240021 A. 102346415,-426.44,NaN
6642,gesa,2024-05-07,VISA APOTHEKE AM VIKT-LUISE,NR XXXX 5021 BERLIN KAUFUMSATZ 04.05 094907 AR...,-402.76,NaN
...,...,...,...,...,...,...
7781,giro,2024-09-17,Getsafe Insurance AG,Dein Getsafe Schadenfall DE-GE-61-1 83368,854.94,NaN
6294,gesa,2024-03-26,NORGES FORSKNINGSRAD,REF.-NR. ING-DIBA REF20 2024032623007909 EUR 9...,978.46,NaN
7063,gesa,2024-06-27,NORGES FORSKNINGSRAD,REF.-NR. ING-DIBA REF20 2024062725083091 EUR 1...,1135.22,NaN
6959,gesa,2024-06-13,Mathias Karle Gastehaus Kaiserpassage,Ruckerstattung AZ Angebot 178857 Ga stehaus Ka...,1416.00,NaN


In [ ]:
## Innerhalb der nicht-kategorisierten Abbuchungen, kumulative Summe der Top-X Beträge berechnen und plotten
import matplotlib.pyplot as plt

# Calculate the cumulative sum of the amounts
filtered_df = filtered_df.sort_values(by='amount', ascending=True)
filtered_df['cumulative_sum'] = filtered_df['amount'].cumsum()

# Plot the cumulative sum
plt.figure(figsize=(20, 12))
plt.plot(range(1, len(filtered_df) + 1), filtered_df['cumulative_sum'])
plt.xlabel('Number of Items')
plt.ylabel('Cumulative Sum of Amount')
plt.title('Cumulative Sum of Top-x Uncategorized Expenses')
plt.grid(True)
plt.show()

In [ ]:
## Gesamteinnahmen 2024
income_df = pc.loc[
    (pc.book_date.dt.year == 2024) &
    (pc['cat'].isin(['einnahmen::gehalt::andreas', 'einnahmen::gehalt::gesa', 'einnahmen::dividende']))
    ]
income_df

In [ ]:
## Einkommensuebersicht 2024
def generate_income_overview(income_df: pd.DataFrame) -> pd.DataFrame:
    # Sum by category
    category_sum = income_df.groupby('cat')['amount'].sum()

    # Overall sum
    overall_sum = income_df['amount'].sum()

    # Combine results into a DataFrame
    overview_df = category_sum.reset_index().rename(columns={'amount': 'category_sum'})
    overview_df.loc[len(overview_df)] = ['Overall Sum', overall_sum]

    return overview_df
# Generate the overview
income_overview_df = generate_income_overview(income_df)
income_overview_df

In [ ]:
## Filter für alle Ausgaben im Jahr 2024
expenses_df = pc.loc[
    (pc.book_date.dt.year == 2024) &  # Nur Buchungen aus dem Jahr 2024
    (~pc['cat'].str.startswith('intern', na=False)) &  # "intern"-Kategorie ausschließen
    (~pc['cat'].str.startswith('einnahmen', na=False)) &  # "einnahmen"-Kategorie ausschließen
    (pc['transfer_category'].isna()) &  # transfer_category muss NaN sein
    (pc['account'].isin(['giro', 'common', 'gesa']))  # Nur bestimmte Konten berücksichtigen
    ]

print(expenses_df['amount'].sum())
expenses_df

In [ ]:
## Ausgaben nach Kategorie, Betragstyp und Konto gruppieren
def generate_expense_overview(expenses_df: pd.DataFrame) -> pd.DataFrame:
    # Replace NaN in 'cat' with 'Uncategorized'
    expenses_df['cat'] = expenses_df['cat'].fillna('Uncategorized')

    # Sum by category and account
    category_account_sum = expenses_df.groupby(['cat', 'account'])['amount'].sum().unstack(fill_value=0)

    # Sum by category across all accounts
    category_sum = expenses_df.groupby('cat')['amount'].sum()

    # Overall sum across all accounts
    overall_sum = expenses_df['amount'].sum()

    # Combine results into a DataFrame
    overview_df = category_sum.reset_index().rename(columns={'amount': 'category_sum'})
    overview_df['giro'] = overview_df['cat'].map(category_account_sum.get('giro', {}))
    overview_df['gesa'] = overview_df['cat'].map(category_account_sum.get('gesa', {}))
    overview_df['common'] = overview_df['cat'].map(category_account_sum.get('common', {}))

    # Add overall sum as a final row
    overall_row = pd.DataFrame([{
        'cat': 'Overall Sum',
        'category_sum': overall_sum,
        'giro': category_account_sum.loc[:, 'giro'].sum() if 'giro' in category_account_sum else 0,
        'gesa': category_account_sum.loc[:, 'gesa'].sum() if 'gesa' in category_account_sum else 0,
        'common': category_account_sum.loc[:, 'common'].sum() if 'common' in category_account_sum else 0,
    }])

    overview_df = pd.concat([overview_df, overall_row], ignore_index=True)

    return overview_df

# Generate the overview
expense_overview_df = generate_expense_overview(expenses_df)
expense_overview_df

In [ ]:
pc[(pc.account == "giro") & (pc.amount > 0) & (pc.book_date.dt.year == 2024)]

In [ ]:
## Anwaltskosten
df['book_date'].min()
df = pc
filtered_df = df[
    df['party'].str.contains('KNH|zirngibl', case=False, na=False) |
    df['purpose'].str.contains('KNH|zirngibl', case=False, na=False)
    ]
filtered_df

In [ ]:
## Anwaltskosten #2
filtered_df = df[
    df['cat'].str.startswith('anwalt', na=False) |
    df['purpose'].str.contains('luig', case=False, na=False) |
    df['party'].str.contains('liu', case=False, na=False)
    ]
filtered_df

In [ ]:
# Alle wohnen::putzen Ausgaben fr das Jahr 2023
df = pc[(pc.account == 'common') & (pc.book_date > datetime.datetime(2023, 2, 1, 0,0,0)) & (pc.book_date < datetime.datetime(2024, 2, 1, 0,0,0))]
filtered_df = df[df['cat'] == 'wohnen::putzen']
filtered_df

In [3]:
# Alle wohnen::putzen Ausgaben fr das Jahr 2024
df = pc[(pc.account == 'common') & (pc.book_date > datetime.datetime(2024, 2, 1, 0,0,0)) & (pc.book_date < datetime.datetime(2025, 2, 1, 0,0,0))]
filtered_df = df[df['cat'] == 'wohnen::putzen']
filtered_df

,account,book_date,valuta_date,party,book_text,purpose,amount,balance,transfer_category,cat
5972,common,2024-02-09,2024-02-09,INES BORNEMANN,Lastschrift,Rg.Nr.29007 Kd.Nr. 2091 Haushalthil fe Januar ...,-696.15,-10.19,NaN,wohnen::putzen
6208,common,2024-03-11,2024-03-11,INES BORNEMANN,Lastschrift,Rg.Nr.29102 Kd.Nr. 2091 Haushalthil fe Februar...,-675.42,767.97,NaN,wohnen::putzen
6432,common,2024-04-11,2024-04-11,INES BORNEMANN,Lastschrift,Rg.Nr.29189 Kd.Nr. 2091 Haushalthil fe Marz AS...,-618.80,447.87,NaN,wohnen::putzen
6693,common,2024-05-14,2024-05-14,INES BORNEMANN,Lastschrift,Rg.Nr.29298 Kd.Nr. 2091 Haushalthil fe April A...,-680.68,580.91,NaN,wohnen::putzen
6948,common,2024-06-12,2024-06-12,INES BORNEMANN,Lastschrift,Rg.Nr.29396 Kd.Nr. 2091 Haushalthil fe Mai ASL...,-716.88,-367.34,NaN,wohnen::putzen
7159,common,2024-07-09,2024-07-09,INES BORNEMANN,Lastschrift,Rg.Nr.29474 Kd.Nr. 2091 Haushalthil fe Juni AS...,-605.81,-467.52,NaN,wohnen::putzen
7457,common,2024-08-13,2024-08-13,INES BORNEMANN,Lastschrift,Rg.Nr.29580 Kd.Nr. 2091 Haushalthil fe Juli AS...,-643.55,1161.59,NaN,wohnen::putzen
7744,common,2024-09-13,2024-09-13,INES BORNEMANN,Lastschrift,Rg.Nr.29663 Kd.Nr. 2091 Haushalthil fe August ...,-531.24,1702.07,NaN,wohnen::putzen
7952,common,2024-10-09,2024-10-09,INES BORNEMANN,Lastschrift,Rg.Nr.29761 Kd.Nr. 2091 Haushalthil fe Septemb...,-618.80,1669.36,NaN,wohnen::putzen
8199,common,2024-11-12,2024-11-12,INES BORNEMANN,Lastschrift,Rg.Nr.29852 Kd.Nr. 2091 Haushalthil fe Oktober...,-598.07,5446.73,NaN,wohnen::putzen


In [15]:
# Arbeitszimmer 2024: Darlehenszinsen
pc[(pc.book_date.dt.year == 2024) & (pc.account == 'common') & (pc.purpose.str.contains('Tilgung', case=False, na=False)) & pc.purpose.str.contains('Leistung')]


,account,book_date,valuta_date,party,book_text,purpose,amount,balance,transfer_category,cat
5643,common,2024-01-02,2024-01-02,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 877...,-1038.34,1267.06,NaN,wohnen::rate
5874,common,2024-01-30,2024-01-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 888...,-1038.34,504.69,NaN,wohnen::rate
6126,common,2024-02-29,2024-02-29,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 890...,-1038.34,1274.84,NaN,wohnen::rate
6337,common,2024-04-02,2024-04-02,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 891...,-1038.34,1878.52,NaN,wohnen::rate
6555,common,2024-04-30,2024-04-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 891...,-1038.34,680.33,NaN,wohnen::rate
6824,common,2024-05-30,2024-05-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 892...,-1038.34,-816.01,NaN,wohnen::rate
7090,common,2024-07-01,2024-07-01,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 893...,-1038.34,827.24,NaN,wohnen::rate
7307,common,2024-07-30,2024-07-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 893...,-1038.34,-920.30,NaN,wohnen::rate
7595,common,2024-08-30,2024-08-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 894...,-1038.34,1699.57,NaN,wohnen::rate
7853,common,2024-09-30,2024-09-30,Andreas Profous Gesa Geisler,Lastschrift,Rechnung Darl.-Leistung 6070166475 Tilgung 895...,-1038.34,-582.13,NaN,wohnen::rate


In [18]:
# Arbeitszimmer 2024: Stromkosten
naturstrom_2024 = pc[pc.party.str.contains('Naturstrom', case=False, na=False) & (pc.book_date.dt.year == 2024)]
naturstrom_2024.amount.sum()

np.float64(-1165.35)

In [24]:
# Arbeitszimmer 2024: Hausgeld
wohngeld = pc[(pc.cat=='wohnen::wohngeld') & (pc.book_date.dt.year == 2024)]
wohngeld.amount.sum()

np.float64(-6659.619999999999)

In [31]:
# Arbeitszimmer 2024: Grundsteuer
grundsteuer = pc[(pc.book_date.dt.year == 2024) & (pc.amount < 0) & (pc.purpose.str.contains('Grundst', case=False, na=False))]
grundsteuer.amount.sum()

np.float64(-346.32)

In [53]:
# Arbeitszimmer 2024: Telefon Mobil (Internet ist auf kontist, und deswegen hier nicht sichtbar)
congstar = pc[(pc.book_date.dt.year == 2024) & (pc.purpose.str.contains('2212684943'))]
congstar.amount.sum()



np.float64(-438.22)